In [3]:
# 矩阵相乘 [28,28] ->(flat) [784] => [1,784]
# X = [v1,v2,...v784]

# H1 = XW1 + b1 [1,784][d1,784]T + [d1] => [1,d1]+[d1]
# W1:[d1,dx]
# b1:[d1]
# H2 = H1W2 + b2
# ....

#Loss?
# H3:[1,d3]
# one-hot eg: 1=> [0,1,0,0,0,0,0,0,0]; 3=> [0,0,0,3,0,0,0,0,0]

# pred = W3 *{W2[W1X+b1]+b2}+b3

# Linear Combination?
# Non-linear Factor ReLU
# H1 = relu(XW1+b1)

In [4]:
# Gradient Descent
# object = (pred-Y) minimize object

#steps load data 
#build model
#tarin
#test

In [5]:
import torch
from matplotlib import pyplot as plt

In [6]:
def plot_curve(data):
    fig = plt.figure()
    plt.plot(range(len(data)),data,color="blue")
    plt.legend(['value'], loc='upper right')
    plt.xlabel('step')
    plt.ylabel('value')
    plt.show()

In [7]:
def plot_image(img, label,name):
    fig = plt.figure()
    for i in range(6):
        plt.subplot(2,3,i+1)
        plt.tight_layout()
        plt.imshow(img[i][0]*0.3081+0.1307, cmap='gray',interpolation='none')
        plt.title("{}:{}".format(name,label[i].item()))
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [8]:
def one_hot(label, depth=10):
    out = torch.zeros(label.size(0), depth)
    idx = torch.LongTensor(label).view(-1,1)
    out.scatter_(dim=1, index=idx,value=1)
    return out

In [9]:
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision

In [10]:
# step1 load dataset
batch_size = 512
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('mnist_data', train=True,download=True,
                              transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize(
                                  (0.1307,),(0.3081,))
                              ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('mnist_data/', train=False,download=True,
                              transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize(
                                  (0.1307,),(0.3081,))
                              ])),
    batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



In [13]:
x,y = next(iter(train_loader))
print(x.shape,y.shape,x.min(),x.max())

torch.Size([512, 1, 28, 28]) torch.Size([512]) tensor(-0.4242) tensor(2.8215)
